In [1]:
import numpy as np
import gdspy
from autograd import numpy as npa

################################################### design parameter #########################
resolution = 50  # Resolution
design_region_width = 3  # Design region width
design_region_height = 4  # Design region height
pml_size = 1  # PML region size
Sx = 1.0  # Height: from top to bottom PML 0 to LN, below that SiO2
Sy = 3.0  # Width: left and right PML, width 3um
Sz = 3.0  # Length (direction of propagation): 3
LNwg_h = 0.5  # LN waveguide
# Design space
design_region_x = LNwg_h
design_region_y = Sy
design_region_z = 2
Nx = int(resolution * design_region_x) + 1
Ny = int(resolution * design_region_y) + 1
Nz = int(resolution * design_region_z) + 1
##############################################################################################

# Load and process structure weight data
structure_weight = np.loadtxt('lastdesign.txt')  # Adjust path as needed
structure_weight = structure_weight.reshape(Nx, Ny * Nz)[Nx - 1]
data = npa.rot90(structure_weight.reshape(Ny, Nz))

# Check if the cell 'TOP' exists and delete it if it does
if 'TOP' in gdspy.current_library.cells:
    del gdspy.current_library.cells['TOP']

# Create the new cell
cell = gdspy.Cell('TOP')

# Rectangle dimensions
width = 1
height = 1
 
 # Numpy 배열을 기반으로 GDS의 경로(Path) 생성
rectangles = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        if data[i, j] == 1:
            # 각 사각형의 좌표를 정밀하게 조정
            x0 = j * width
            y0 = -i * height
            x1 = (j + 1) * width
            y1 = -(i + 1) * height
            rectangle = gdspy.Rectangle((x0, y0), (x1, y1))
            rectangles.append(rectangle)

def chaikin_cutting(points, iterations=1):
    """
    Apply Chaikin's Corner Cutting algorithm to smooth corners of a polygon.
    `points`: list of (x, y) tuples representing the polygon.
    `iterations`: number of times the algorithm should be applied.
    """
    for _ in range(iterations):
        new_points = []
        for i in range(len(points) - 1):
            p0 = points[i]
            p1 = points[i + 1]
            # Calculate new points at 1/4 and 3/4 positions between p0 and p1
            q = (0.75 * p0[0] + 0.25 * p1[0], 0.75 * p0[1] + 0.25 * p1[1])
            r = (0.25 * p0[0] + 0.75 * p1[0], 0.25 * p0[1] + 0.75 * p1[1])
            new_points.extend([q, r])
        new_points.append(points[-1])  # Append the last point
        points = new_points
    return points

if rectangles:
    # 병합된 다각형을 생성
    # 직사각형을 살짝 확장하여 작은 간격을 메움
    expanded_rectangles = [gdspy.offset(rectangle, distance=0.05) for rectangle in rectangles]

    # 확장된 직사각형들을 병합
    merged_polygon = gdspy.boolean(expanded_rectangles, None, 'or', precision=1e-6, max_points=500)

    # 병합된 다각형이 존재할 때 처리
    if merged_polygon:
        for polygon_boundary in merged_polygon.polygons:
            # Chaikin's algorithm을 사용해 모서리만 부드럽게 처리
            smoothed_points = chaikin_cutting(polygon_boundary, iterations=2)
            smoothed_polygon = gdspy.Polygon(smoothed_points)

            # 매끄럽게 처리된 다각형을 셀에 추가
            cell.add(smoothed_polygon)

# Save the GDS file
gdspy.write_gds('output.gds')

# Optionally, save an image of the cell as SVG.
cell.write_svg('output.svg')

# Display all cells using the internal viewer.
gdspy.LayoutViewer()



/tmp/ipykernel_1978082/3321370011.py:96: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>